# Random Forest

Training:
- Get a random subset of the dataset
- Create a decision tree with this random subset.
- Repeat for as many times as the number of trees that we want.

Testing (given a datapoint we want to test)
- Get the predictions from each tree
- Hold a majority vote

In [ ]:
import numpy as np
from Collections import Counter

In [ ]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None,*, value=None): #value will only be passed to leaf nodes
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None

class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_features = n_features
        self.root = None

    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1], self.n_features)
        self.root = self._grow_tree(X,y)

    # Recursive function to grow the tree
    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))
        # Check if we have reached the stopping criteria

        if (depth >= self.max_depth or
            n_samples < self.min_samples_split or
            n_labels == 1):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        # Find the best split
        best_thresh, best_feature = self._best_split(X, y, feat_idx)
        # Create the left and right subtrees


    def most_common_label(self, y):
        # Return the most common label in y
        counter = Counter(y)
        most_common = counter.most_common(1)[0][0]
        return most_common

    def predict():

In [ ]:
class RandomForest:
    def __init__(self, n_trees=100, max_depth=10, min_samples_split=2, n_features=None):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.n_features = n_features
        self.trees = []


    def fit():

    
    def predict():
        